This is by no means a comprehensive example of all the torch_geometric(PyG) can do, but it illustrates some of it's capabilites

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data# this contains our dataset
import numpy as np
from torch_geometric.datasets import Planetoid# this helps us manipulate the dataset
from torch_geometric.nn import GCNConv# this is a simple implementation of the GCN convolutional layer
import torch_geometric.transforms as T

c:\Users\brown\miniconda3\envs\mainenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class GCN(torch.nn.Module):# this is the torch geometric implementation of our GCN model like before, it
    # is a lot simpler to implement and way customizeable
    def __init__(self, in_feat, hid_feat, out_feat):
        super().__init__()
        self.conv1 = GCNConv(in_feat, hid_feat)
        self.conv2 = GCNConv(hid_feat, out_feat)
        self.activation = nn.ReLU()
        #self.dropout = nn.Dropout(p=.4)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.activation(self.conv1(x, edge_index))
        x = F.dropout(x, training= self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x,dim=1)

In [11]:
transform = T.Compose([T.RandomNodeSplit(num_val=1000, num_test=1500),
    T.TargetIndegree(),])# this is what is called a transform, basically it is a way of "transforming" our data. In this case it 
                        # randomizes the nodes and creates a training/validation/testing split
dataset = Planetoid(root="tmp/Cora", name="Cora",transform=transform)# this grabs the cora dataset, already well ordered so we do not need a 
# utils module
data = dataset[0]# this just grabs the first graph from the dataset(some datasets train on several graphs, ours just uses one)
model = GCN(dataset.num_node_features,16,dataset.num_classes).to("cuda:0")# this initializes our model and puts it on the GPU(change if you do not have a gpu)
data.to("cuda:0")# puts our data on the GPU as well
optimizer = torch.optim.Adam(params=model.parameters(),lr = .01, weight_decay=5e-3)# sets up our optimizer

In [16]:
def accuracy(preds,mask):# obtains the accuracy of the model
    correct = (preds[mask] == data.y[mask]).sum()
    acc = int(correct)/int(mask.sum())
    return acc

In [18]:
model.train()# tells our model we are about to train
for epoch in range(200):# runs through all the data 200 times
    optimizer.zero_grad()
    out = model(data)

    train_loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    train_acc = accuracy(out.max(1)[1], data.train_mask)

    val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask])
    val_acc = accuracy(out.max(1)[1], data.val_mask)

    if(epoch %5 == 0):
        print("epoch: {} training_loss: {} training_acc: {} val_loss: {} val_acc: {}".format(epoch,train_loss,train_acc,val_loss, val_acc))
        
    train_loss.backward()
    optimizer.step()
model.eval()
preds = model(data).max(1)[1]
acc = accuracy(preds,data.test_mask)
print(acc)

epoch: 0 training_loss: 0.1681639403104782 training_acc: 0.9855769230769231 val_loss: 0.6935117244720459 val_acc: 0.786
epoch: 5 training_loss: 0.15806561708450317 training_acc: 0.9807692307692307 val_loss: 0.6871837377548218 val_acc: 0.772
epoch: 10 training_loss: 0.1618833839893341 training_acc: 0.9759615384615384 val_loss: 0.7041781544685364 val_acc: 0.782
epoch: 15 training_loss: 0.13891062140464783 training_acc: 0.9903846153846154 val_loss: 0.6866135597229004 val_acc: 0.785
epoch: 20 training_loss: 0.16179326176643372 training_acc: 0.9951923076923077 val_loss: 0.6573135852813721 val_acc: 0.791
epoch: 25 training_loss: 0.13362950086593628 training_acc: 0.9807692307692307 val_loss: 0.6789388656616211 val_acc: 0.792
epoch: 30 training_loss: 0.15979844331741333 training_acc: 0.9855769230769231 val_loss: 0.6920156478881836 val_acc: 0.794
epoch: 35 training_loss: 0.14456354081630707 training_acc: 0.9903846153846154 val_loss: 0.6908521056175232 val_acc: 0.782
epoch: 40 training_loss: 0.1